In [3]:
using JuMP
using Ipopt

In [37]:
model = Model(Ipopt.Optimizer)
set_silent(model)

num_circles = 1:2

@variable(model, x[num_circles], start = 1) #<-- the variable's name is defined in the second argument
@variable(model, y[num_circles], start = 1)
# @variable(model, coords[num_circles, 1:2], start = 1) #<-- this would produce a matrix 
@NLparameter(model, r[i = num_circles] == i)
# Using NLparameter allows us to change model params without having to remake the model

squared_diff = @NLexpression(model, [i = num_circles], (x[i]*x[i] + y[i]*y[i] - r[i]*r[i])^2)

sum_diff = @NLexpression(model, sum(squared_diff[i] for i=num_circles))

@NLconstraint(model, intersection[i=num_circles], x[i] == 2*y[i])
#@NLconstraint(model, [i=num_circles], x[i] == 2*y[i]) <-- constraints can be anonymous

@NLobjective(model, Min, sum_diff)

print(model)
optimize!(model)

for circle in num_circles
    circle_r = value.(r)[circle]
    circle_constraint = intersection[circle]
    solution_x = round((value.(x)[circle]), digits = 3)
    solution_y = round((value.(y)[circle]), digits = 3)
    print("For circle with radius $circle_r and constraint $circle_constraint, solution x = $solution_x, y = $solution_y")
    print("\n")
end
print("\n")
print("The output below shows the raw solutions stored internally:\n\n")

@show value.(x)
@show value.(y)


For circle with radius 1.0 and constraint x[1] - 2.0 * y[1] = 0, solution x = 0.894, y = 0.447
For circle with radius 2.0 and constraint x[2] - 2.0 * y[2] = 0, solution x = 1.789, y = 0.894

The output below shows the raw solutions stored internally:

value.(x) = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:2
And data, a 2-element Vector{Float64}:
 0.8944271909999159
 1.788854381999832
value.(y) = 1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:2
And data, a 2-element Vector{Float64}:
 0.4472135954999579
 0.894427190999916


1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:2
And data, a 2-element Vector{Float64}:
 0.4472135954999579
 0.894427190999916